<a href="https://colab.research.google.com/github/takky0330/Arduino/blob/master/BERT_livedoor_news_IIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [6]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

In [7]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2020-05-22 05:20:58--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.98MB/s    in 2.8s    

2020-05-22 05:21:02 (2.98 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [8]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)

['peachy', 'it-life-hack', 'livedoor-homme', 'README.txt', 'sports-watch', 'dokujo-tsushin', 'CHANGES.txt', 'topic-news', 'smax', 'movie-enter', 'kaden-channel']
カテゴリー数: 9
['peachy', 'it-life-hack', 'livedoor-homme', 'sports-watch', 'dokujo-tsushin', 'topic-news', 'smax', 'movie-enter', 'kaden-channel']


In [0]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text

In [0]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)

In [11]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

(7376, 2)


In [0]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]

In [0]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)

In [14]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)

(1475, 2)
(5901, 2)


In [0]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")

In [16]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [17]:
import torch
import torchtext  # torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')

In [0]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定

In [0]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

In [20]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

4426
1475
1475


In [0]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}

In [22]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
model.eval()
print('ネットワーク設定完了')


ネットワーク設定完了


In [0]:
# BERTでベクトル化する関数を定義


def vectorize_with_bert(net, dataloader):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloader.batch_size

    # データローダーからミニバッチを取り出すループ
    for index, batch in enumerate(dataloader):
        # batchはTextとLableの辞書オブジェクト
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs = batch.Text[0].to(device)  # 文章
        labels = batch.Label.to(device)  # ラベル

        # 順伝搬（forward）計算
        with torch.set_grad_enabled(False):

            # Berに入力
            result = net(inputs)

            # sequence_outputの先頭の単語ベクトルを抜き出す
            vec_0 = result[0]  # 最初の0がsequence_outputを示す
            vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
            vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換

            # ベクトル化したデータをtorchリストにまとめる
            if index == 0:
                list_text = vec_0
                list_label = labels
            else:
                list_text = torch.cat([list_text, vec_0], dim=0)
                list_label = torch.cat([list_label, labels], dim=0)

    return list_text, list_label

In [24]:
# DataLoaderをベクトル化版に変換
# 少し時間がかかります5分弱

list_text_train, list_label_train = vectorize_with_bert(model, dl_train)
list_text_eval, list_label_eval = vectorize_with_bert(model, dl_eval)
list_text_test, list_label_test = vectorize_with_bert(model, dl_test)

使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------


In [0]:
# torchのリストをDatasetに変換

from torch.utils.data import TensorDataset

dataset_bert_train = TensorDataset(
    list_label_train.view(-1, 1), list_text_train)
dataset_bert_eval = TensorDataset(list_label_eval.view(-1, 1), list_text_eval)
dataset_bert_test = TensorDataset(list_label_test.view(-1, 1), list_text_test)

In [0]:
# Dataloaderにする
from torch.utils.data import DataLoader

batch_size = 1024

dl_bert_train = DataLoader(
    dataset_bert_train, batch_size=batch_size, shuffle=True, drop_last=True)
# drop_lastは最後のミニバッチがbatch_sizeに足りない場合は無視する

dl_bert_eval = DataLoader(
    dataset_bert_eval, batch_size=batch_size, shuffle=False)
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=batch_size, shuffle=False)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

OVER_CLUSTRING_RATE = 10


class NetIIC(nn.Module):
    def __init__(self):
        super(NetIIC, self).__init__()

        # multi-headは今回しない
        self.conv1 = nn.Conv1d(1, 400, kernel_size=768, stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(400)
        self.conv2 = nn.Conv1d(1, 300, kernel_size=400, stride=1, padding=0)
        self.bn2 = nn.BatchNorm1d(300)
        self.conv3 = nn.Conv1d(1, 300, kernel_size=300, stride=1, padding=0)
        self.bn3 = nn.BatchNorm1d(300)

        self.fc1 = nn.Linear(300, 250)
        self.bnfc1 = nn.BatchNorm1d(250)

        # livedoorニュースの9カテゴリに対応するかな？と期待する9分類
        self.fc2 = nn.Linear(250, 9)

        # overclustering
        # 実際の想定よりも多めにクラスタリングさせることで、ネットワークで微細な変化を捉えられるようにする
        self.fc2_overclustering = nn.Linear(250, 9*OVER_CLUSTRING_RATE)

    def forward(self, x):
        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn1(self.conv1(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn2(self.conv2(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn3(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x_prefinal = F.relu(self.bnfc1(self.fc1(x)))

        # multi-headは使わず
        y = F.softmax(self.fc2(x_prefinal), dim=1)
        y_overclustering = F.softmax(self.fc2_overclustering(
            x_prefinal), dim=1)  # overclustering

        return y, y_overclustering

In [0]:
import torch.nn.init as init


def weight_init(m):
    """重み初期化"""
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        # Xavier
        # init.xavier_normal_(m.weight.data)

        # He
        init.kaiming_normal_(m.weight.data)

        if m.bias is not None:
            init.normal_(m.bias.data)

In [0]:
# IISによる損失関数の定義
# 参考：https://github.com/RuABraun/phone-clustering/blob/master/mnist_basic.py
import sys


def compute_joint(x_out, x_tf_out):
    bn, k = x_out.size()
    assert (x_tf_out.size(0) == bn and x_tf_out.size(1) == k), '{} {} {} {}'.format(
        bn, k, x_tf_out.size(0), x_tf_out.size(1))

    p_i_j = x_out.unsqueeze(2) * x_tf_out.unsqueeze(1)  # bn, k, k
    p_i_j = p_i_j.sum(dim=0)  # k, k
    p_i_j = (p_i_j + p_i_j.t()) / 2.  # symmetrise
    p_i_j = p_i_j / p_i_j.sum()  # normalise
    return p_i_j


def IID_loss(x_out, x_tf_out, EPS=sys.float_info.epsilon):
    # has had softmax applied
    bs, k = x_out.size()
    p_i_j = compute_joint(x_out, x_tf_out)
    assert (p_i_j.size() == (k, k))

    p_i = p_i_j.sum(dim=1).view(k, 1).expand(k, k)
    p_j = p_i_j.sum(dim=0).view(1, k).expand(k, k)

    # avoid NaN losses. Effect will get cancelled out by p_i_j tiny anyway
    # これはPyTorchのバージョン1.3以上だとエラーになる
    # https://discuss.pytorch.org/t/pytorch-1-3-showing-an-error-perhaps-for-loss-computed-from-paired-outputs/68790/3
    #p_i_j[(p_i_j < EPS).data] = EPS
    #p_j[(p_j < EPS).data] = EPS
    #p_i[(p_i < EPS).data] = EPS

    p_i_j = torch.where(p_i_j < EPS, torch.tensor(
        [EPS], device=p_i_j.device), p_i_j)
    p_j = torch.where(p_j < EPS, torch.tensor([EPS], device=p_j.device), p_j)
    p_i = torch.where(p_i < EPS, torch.tensor([EPS], device=p_i.device), p_i)

    # https://qiita.com/Amanokawa/items/0aa24bc396dd88fb7d2a
    # 参考に、重みalphaを追加

    alpha = 2.0
    loss = (- p_i_j * (torch.log(p_i_j) - alpha *
                       torch.log(p_j) - alpha*torch.log(p_i))).sum()

    return loss

In [0]:
# データにノイズを加える関数の定義
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor_std = list_text_train.std(dim=0).to(device)


def perturb_data(x):
    y = x.clone()
    noise = torch.randn(len(tensor_std)).to(device)*tensor_std*2.0
    noise = noise.expand(x.shape[0], -1)
    y += noise

    return y

In [0]:
# 学習関数の定義

def train(total_epoch, model, train_loader, optimizer, device):

    # ネットワークを訓練モードに
    model.train()

    # 学習率のスケジューラーCosAnnealing
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=2, T_mult=2, eta_min=0)

    for epoch in range(total_epoch):
        for batch_idx, (target, data) in enumerate(train_loader):

            # 学習率変化
            scheduler.step()
            
            data_perturb = perturb_data(data)  # ノイズを与え、変換したデータを作る

            # GPUに送れる場合は送る
            data = data.to(device)
            data_perturb = data_perturb.to(device)

            # 最適化関数の初期化
            optimizer.zero_grad()

            # ニューラルネットワークへ入れる
            output, output_overclustering = model(data)
            output_perturb, output_perturb_overclustering = model(data_perturb)

            # 損失の計算
            loss1 = IID_loss(output, output_perturb)
            loss2 = IID_loss(output_overclustering,
                             output_perturb_overclustering)
            loss = loss1 + loss2

            # 損失を減らすように更新
            loss.backward()
            optimizer.step()

        # ログ出力
        if epoch % 50 == 0:
            print('Train Epoch {} \tLoss1: {:.6f} \tLoss2: {:.6f} \tLoss_total: {:.6f}'.format(
                epoch, loss1.item(), loss2.item(), loss1.item()+loss2.item()))

    return model, optimizer

In [32]:
# 学習の実施(5分弱)

# モデルの用意
net = NetIIC()
net.apply(weight_init)
net.to(device)

# 最適化関数
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4) 

total_epoch = 1000

model_trained, optimizer = train(
    total_epoch, net, dl_bert_train, optimizer, device)

Train Epoch 0 	Loss1: -3.944716 	Loss2: -7.745652 	Loss_total: -11.690368
Train Epoch 50 	Loss1: -6.165148 	Loss2: -10.256249 	Loss_total: -16.421398
Train Epoch 100 	Loss1: -6.493257 	Loss2: -11.206728 	Loss_total: -17.699985
Train Epoch 150 	Loss1: -6.562608 	Loss2: -11.715666 	Loss_total: -18.278274
Train Epoch 200 	Loss1: -6.560358 	Loss2: -12.417967 	Loss_total: -18.978324
Train Epoch 250 	Loss1: -6.552682 	Loss2: -12.592127 	Loss_total: -19.144809
Train Epoch 300 	Loss1: -6.566082 	Loss2: -13.142930 	Loss_total: -19.709012
Train Epoch 350 	Loss1: -6.562592 	Loss2: -13.185946 	Loss_total: -19.748537
Train Epoch 400 	Loss1: -6.542658 	Loss2: -13.153091 	Loss_total: -19.695749
Train Epoch 450 	Loss1: -6.567782 	Loss2: -13.166708 	Loss_total: -19.734490
Train Epoch 500 	Loss1: -6.567127 	Loss2: -13.224188 	Loss_total: -19.791315
Train Epoch 550 	Loss1: -6.557654 	Loss2: -13.164039 	Loss_total: -19.721693
Train Epoch 600 	Loss1: -6.575311 	Loss2: -13.198984 	Loss_total: -19.774295
Tra

In [0]:
# モデル分類のクラスターの結果を確認する
import numpy as np

# ミニバッチサイズ1のテスト用のDataLoaderを用意
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=1, shuffle=False)


def test(model, device, test_loader):
    model.eval()

    out_targs = []
    ref_targs = []

    # 出力用のリストを用意
    total_num = len(test_loader)
    # index, (target_label, inferenced_label)
    output_list = np.zeros((total_num, 2))

    with torch.no_grad():
        for batch_idx, (target, data) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            outputs, outputs_overclustering = model(data)

            # 分類結果をリストに追加
            out_targs.append(outputs.argmax(dim=1).cpu())
            ref_targs.append(target[0].cpu())

            # 結果をリストにまとめる
            output_list[batch_idx, 0] = target[0][0].cpu()  # 正解ラベル
            output_list[batch_idx, 1] = outputs.argmax(dim=1).cpu()  # 予測ラベル

    out_targs = torch.cat(out_targs)
    ref_targs = torch.cat(ref_targs)

    return out_targs.view(-1, 1).numpy(), ref_targs.numpy(), output_list

In [0]:
# テストデータで推論を実施
out_targs, ref_targs, output_list = test(model_trained, device, dl_bert_test)

In [35]:
# 混同行列（的な）を作る
matrix = np.zeros((9, 9))

# 縦にlivedoorニュースの正解クラスを、横に判定されたクラスの頻度表を作成
for i in range(len(out_targs)):
    row = ref_targs[i]
    col = out_targs[i]
    matrix[row][col] += 1

np.set_printoptions(suppress=True)
print(matrix)

[[ 13.  24.  14.   2.  32.  12.  50.   1.  30.]
 [ 20.   2.  61.  32.   4.  16.  38.   0.   1.]
 [  7.   3.  17.   5.  34.   4.  24.   0.   8.]
 [ 57.   1.   2.   0.   0.  46.   2.  59.   1.]
 [  2.  11.   3.   1.  14.   0.   2.   1. 133.]
 [ 30.   2.   2.   0.   4.  64.   2.  49.   1.]
 [  5.   2.  11. 103.   0.  10.  42.   0.   1.]
 [  9. 145.   7.   2.   2.   6.   3.   4.   3.]
 [ 68.   5.  29.   9.   7.  37.  17.   3.   2.]]


In [36]:
dic_id2cat

{0: 'peachy',
 1: 'it-life-hack',
 2: 'livedoor-homme',
 3: 'sports-watch',
 4: 'dokujo-tsushin',
 5: 'topic-news',
 6: 'smax',
 7: 'movie-enter',
 8: 'kaden-channel'}

In [47]:
# クラスタの結果を確認
#「sports-watch」の5番目のクラスタの文章、7番目のクラスタの文章
#「topic-news」の5番目のクラスタの文章、7番目のクラスタの文章
# を確認して、クラスタ5とクラスタ7の特徴を見てみます。

import pandas as pd

df2 = pd.DataFrame(output_list)
df2.columns=["正解クラス", "推定クラスタ"]
df2.head()

,正解クラス,推定クラスタ
0,5.0,7.0
1,6.0,3.0
2,3.0,0.0
3,6.0,3.0
4,5.0,5.0


In [48]:
df2[(df2['正解クラス']==0) & (df2['推定クラスタ']==5)].head()

,正解クラス,推定クラスタ
48,0.0,5.0
492,0.0,5.0
563,0.0,5.0
704,0.0,5.0
830,0.0,5.0


In [49]:
# dfに元文書を入れている。300文字ほど見る。
print(df.iloc[21, 0][:300])
print(df.iloc[59, 0][:300])

オーストラリアで最も愛されるワインブランド「ジェイコブス・クリーク」では、春の行楽シーズンに合わせて「Enjoy Wine Outdoors Campaign」を5月31日（木）まで開催中。オージー・ビーフとワインで楽しむ大人のアウトドアタイムを提案します。ジェイコブス・クリークのスティル（非発砲性）ワインは一部を除き全てスクリューキャップなので、専用のオープナーがなくても開けられ、アウトドアにぴったりです。キャンペーン期間中、対象商品のコア・レンジ8品とリザーブ・レンジ5品にもれなく、快適生活研究家 田中ケンさんが考案した「オージー・ビーフを使ったアウトドアクッキングレシピ」をオンパック。赤
2009年に小説が発売されて反響を呼んだ『アントキノイノチ』は、生きることに絶望した男女が遺品整理業を通して未来に踏み出していく姿を描いた感動作です。今最も注目を集めている俳優陣と第61回ベルリン国際映画祭で2冠を遂げた瀬々敬久監督により、ついに映画化されることになりました。主演は、若手実力派俳優として名高い岡田将生と『余命１ヶ月の花嫁』での演技が高く評価された榮倉奈々です。岡田将生演じる永島杏平は、親友の死をきっかけに心を閉ざしてしまった青年。父親に紹介されて、遺品整理会社に勤め始めました。そこで働いていたのが、榮倉奈々演じる久保田ゆきです。ゆきは過去の暴行事件を境に自傷行為を繰り返すように


In [50]:
df2[(df2['正解クラス']==0) & (df2['推定クラスタ']==7)].head(2)

,正解クラス,推定クラスタ
44,0.0,7.0


In [51]:
df2[(df2['正解クラス']==8) & (df2['推定クラスタ']==5)].head(2)

,正解クラス,推定クラスタ
55,8.0,5.0
116,8.0,5.0
